# Test MongoDB
El objetivo de este Jupyter notebook es testear que la base de datos MongoDB esté correctamente instalada, creada completa con la última actualización de los datos que has corrido.

Para ello, ejecutaremos los siguientes tests:
1. Conexión a la base de datos MongoDB en local.
2. Revisar el tamaño y la última actualización de los datos.
3. Crear una muestra pequeña de datos para confirmar que los comandos funcionan.

Nota: La base de datos original tiene más de 24GB, por lo que cada request a la base de datos puede demorar mucho tiempo si no se ejecuta en un sistema en paralelo.

In [1]:
import pymongo
from pymongo import MongoClient

### 1. Conexión a la base de datos MongoDB en local.

In [2]:
# Reemplaza 'localhost' con la dirección de tu servidor MongoDB si es diferente
# y '27017' con el puerto correspondiente si has configurado uno distinto.
client = MongoClient('mongodb://localhost:27017/')

# Reemplaza 'nombre_de_tu_base_de_datos' con el nombre de tu base de datos
db = client['Contrataciones']

# Intenta obtener una colección para verificar la conexión
try:
    # Reemplaza 'nombre_de_tu_colección' con el nombre de una colección existente en tu base de datos
    colecciones = db.list_collection_names()  # Lista todas las colecciones para verificar la conexión
    if 'Contratos_EDCA' in colecciones:
        print("Conexión exitosa a la base de datos y la colección encontrada.")
    else:
        print("Conexión exitosa a la base de datos, pero la colección no fue encontrada.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")


# Listar todas las bases de datos
databases = client.list_database_names()

# Imprimir la lista de bases de datos
print("Bases de datos disponibles son:", databases)

Conexión exitosa a la base de datos y la colección encontrada.
Bases de datos disponibles son: ['Contrataciones', 'Test', 'admin', 'config', 'local']


### 2. Revisar el tamaño y la última actualización de los datos.

In [3]:
# Imprimir el tamaño de la base de datos, número de colecciones y última actualización

# Obteniendo el tamaño de la base de datos en GB
print(f"Obteniendo el tamaño de la base de datos.")
db_stats = db.command("dbstats")
db_size_gb = db_stats['storageSize'] / (1024**3)  # Convertir bytes a GB
print(f"Tamaño de la base de datos: {db_size_gb} GB")

# Obteniendo el número de documentos en la colección 'Contratos_EDCA'
print(f"Obteniendo el número de documentos. Este proceso puede tardar hasta 3 minutos...")
num_documents = db['Contratos_EDCA'].count_documents({})
print(f"Número de documentos en la colección 'Contratos_EDCA': {num_documents}")

# Obteniendo la última actualización de la base de datos
# Esto asume que hay una colección que registra las actualizaciones
# Reemplaza 'nombre_de_tu_colección_de_registro' con el nombre real de tu colección de registro

#  try: [HAY QUE CAMBIAR ESTO POR LA FECHA DEL ÚLTIMO CONTRATO]
#     last_update_collection = db['Contratos_EDCA']
#     last_update = last_update_collection.find().sort("fecha", -1).limit(1)
#     for update in last_update:
#         print(f"Última actualización: {update['fecha']}")
# except Exception as e:
#     print(f"No se pudo obtener la última actualización: {e}")


Obteniendo el tamaño de la base de datos.
Tamaño de la base de datos: 7.33831787109375 GB
Obteniendo el número de documentos. Este proceso puede tardar hasta 3 minutos...
Número de documentos en la colección 'Contratos_EDCA': 2742266


### 3. Crear una muestra pequeña de datos para confirmar que los comandos funcionan.

Guardaremos la nueva muestra _Sample_Contratos_EDCA_ en una nueva colección.

In [10]:
# Generando la muestra aleatoria y guardándola en una nueva colección
sample_size = int(num_documents * 0.05)  # Calcula el 5% del total de documentos
print(f"Número de documentos en la muestra aleatoria es: {sample_size}")

# Realiza la agregación para obtener la muestra aleatoria
sample = db['Contratos_EDCA'].aggregate([
    { '$sample': { 'size': sample_size } }
])
sample_documents = list(sample)
print(f"Sample creado")

# Guarda la muestra en una nueva colección
db['Sample_Contratos_EDCA'].insert_many(sample_documents)
print(f"Muestra guardada en la colección 'Sample_Contratos_EDCA'!")



Número de documentos en la muestra aleatoria es: 137113
Sample creado
Muestra guardada en la colección 'Sample_Contratos_EDCA'!


In [19]:
# Realizando la consulta con sort y limit
resultados = Sample_Contratos_EDCA.find().sort('publishedDate', -1).limit(5)

# Iterando sobre los resultados y haciendo algo con ellos
for documento in resultados:
    print(documento)

AttributeError: 'CommandCursor' object has no attribute 'find'